In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import json
import numpy as np
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

In [ ]:
prospect_to_docs = {}
prospect_to_position = {}

#page of the top 100 prospects
profile_urls = set()
url = "http://www.draftexpress.com/rankings/Top-100-Prospects/#list"
    
r = requests.get(url)
data = r.text
    
soup = BeautifulSoup(data, "html.parser")
    
profile_urls = {tag["href"] for tag in soup.find_all(name="a", href=re.compile(r"\/profile\/.+")) 
                if "/stats" not in tag["href"] and "/videos" not in tag["href"]}

for profile_url in profile_urls:
    player_name = re.findall(r"\/profile\/([^\/]+)", profile_url)[0]
    player_name = " ".join(re.split(r"-", player_name)[:-1])
    print(player_name)

    full_profile_url = "http://www.draftexpress.com" + str(profile_url)
    profile_r = requests.get(full_profile_url)
    profile_data = profile_r.text

    profile_soup = BeautifulSoup(profile_data, "html.parser")
    
    m = re.search('Position:\s(\S+)', profile_soup.get_text())
    prospect_to_position[player_name] = m.group(1)
#     for div in profile_soup.find_all(class_="data small-12 medium-6 column"):
#         m = re.search('Position:\s(\S+)', div.get_text())
#         if m:
#             prospect_to_position[player_name] = m.group(1)
#             break

    text = ""
    for a in profile_soup.find_all(class_="article-content"):    
        cleaned = re.sub(r'<.+>', '', a.get_text())
        cleaned = re.sub(r'<div>.+<\/div>', '', cleaned)
        cleaned = re.sub(r"\([\S\s]+\)", "", cleaned)
        cleaned = cleaned.replace("-", " ")
        cleaned = cleaned.replace("#", "")
        cleaned = cleaned.replace("%", "")
        cleaned = cleaned.replace("Please enable Javascript to watch this video", "")
        text += cleaned
        #if a.parent.name == "div" and "item" in p.parent["class"]:
        #    text.append(a.get_text())

    prospect_to_docs[player_name] = text.encode("utf-8")

with open("prospect_to_docs.json", "w") as f:
    json.dump(prospect_to_docs, f)
    
with open("prospect_to_position.json", "w") as f:
    json.dump(prospect_to_position, f)

In [ ]:
sw = set(stopwords.words('english'))
sw.add("he's")

prospect_words = {}
prospect_lengths = []
with open("prospect_to_docs.json") as f:
    for line in f:
        json_line = json.loads(line)
        all_texts = json_line.values()
        for text in all_texts:
            text = text.replace("Please enable Javascript to watch this video", "")
            tokens = text.split()
            prospect_lengths.append(len(tokens))
            for token in tokens:
                token = token.lower()
                if token not in sw:
                    if token in prospect_words:
                        prospect_words[token] += 1
                    else:
                        prospect_words[token] = 1
print sorted(prospect_words.items(), key= lambda x:x[1], reverse=True)[:50]

print np.mean(prospect_lengths)
plt.hist(prospect_lengths, bins=20, rwidth=0.9)
plt.title("Document Length Histogram for Prospect Articles")
plt.xlabel("Document Length (Number of Tokens)")
plt.ylabel("Number of Documents")
plt.show()

In [ ]:
plt.hist(prospect_words.values(), bins=30, rwidth=0.9, log=True)
plt.title("Word Frequency Histogram for Prospect Articles")
plt.xlabel("Word Count")
plt.ylabel("Frequency")
plt.show()

In [ ]:
player_words = {}
player_lengths = []
with open("curr_player_to_docs.json") as f:
    for line in f:
        json_line = json.loads(line)     
        
        all_texts = json_line.values()

        for text in all_texts:
            text = text.replace("Please enable Javascript to watch this video", "")
            tokens = text.split()
            player_lengths.append(len(tokens))
            for token in tokens:
                token = token.lower()
                if token not in sw:
                    if token in player_words:
                        player_words[token] += 1
                    else:
                        player_words[token] = 1
print sorted(player_words.items(), key= lambda x:x[1], reverse=True)[:50]

print np.mean(player_lengths)
print len(player_lengths)
plt.hist(player_lengths, bins=20, rwidth=0.9)
plt.title("Document Length Histogram for Player Articles")
plt.xlabel("Document Length (Number of Tokens)")
plt.ylabel("Number of Documents")
plt.show()

In [ ]:
plt.hist(player_words.values(), bins=30, rwidth=0.9, log=True)
plt.title("Word Frequency Histogram for Player Articles")
plt.xlabel("Word Count")
plt.ylabel("Frequency")
plt.show()